This file needs requires external data!

(Run on workstation)

In [1]:
import xarray as xr
import glob
import numpy as np

import sys

sys.path.append('../src')
import utils as ut
from utils2 import save_basinvolumes
np.set_printoptions(precision=2)

In [2]:
#Get basin volume
try:
    ds = xr.open_dataset('../data/basinvolumes.nc')
except:
    save_basinvolumes()
    ds = xr.open_dataset('../data/basinvolumes.nc')
volume = ds['volume']
ds.close()

In [3]:
def get_fnames(run):
    if run == 'ctrl':
        fnames = sorted(glob.glob(f'../data/ecefiles/n011/n011*T.nc'))[38:-5]
    elif run == 'spin':
        fnames = sorted(glob.glob(f'../data/ecefiles/n011/n011*T.nc'))[:38]
    else:
        fnames = sorted(glob.glob(f'../data/ecefiles/{run}/{run}*T.nc'))
    return fnames

In [4]:
#Monthly time series

def save_monthly(run):
    fnames = get_fnames(run)

    nmonths = 0
    for fname in fnames:
        ds = xr.open_dataset(fname)
        nmonths += len(ds.time_counter)
        ds.close()
    print(run,':',nmonths, 'months = ',int(np.floor(nmonths/12)),'y',int(nmonths-12*np.floor(nmonths/12)),'m')

    #Calculate basin-average annual time series
    tbas = np.zeros((nmonths,len(volume.basin)))
    ttime = np.arange(nmonths)

    c = 0
    for f,fname in enumerate(fnames):
        ds = xr.open_dataset(fname)
        time = ds['time_centered'].values
        temp = ds['thetao'].values
        ds.close()

        for m in range(len(time)):
            for b,bas in enumerate(volume.basin):
                tbas[c,b] = np.nansum(temp[m,:,:,:]*volume.sel(basin=bas))/np.nansum(volume.sel(basin=bas))
            print(f'{c:3.0f}',tbas[c,:],end='                                                \r')
            c += 1

    #Save data
    temp2 = xr.DataArray(tbas,dims=('time','basin'),coords={'time':ttime,'basin':volume.basin},attrs={'unit':'degrees Celcius','long_name':'temperature time series per basin'})
    ds = xr.Dataset({'temp':temp2})
    savename = f'../data/temperature_mon_{run}.nc'
    ds.to_netcdf(savename,mode='w')
    ds.close()
    print('Saved',savename,end='                     \n')

In [5]:
#Annual time series

def save_annual(run):
    fnames = get_fnames(run)

    nyears = 0
    for fname in fnames:
        ds = xr.open_dataset(fname)
        nyears += int(np.floor(len(ds.time_counter)/12))
        ds.close()
    print(run,':',nyears, 'years')

    #Get seconds per month
    ds = xr.open_dataset('../data/ecefiles/n011/n011_1m_18500101_18501231_grid_T.nc')
    time_bnds = ds['time_centered_bounds']
    ds.close()
    secs = (time_bnds[:12,1]-time_bnds[:12,0]).values / np.timedelta64(1, 's')
    
    #Calculate basin-average annual time series
    tbas = np.zeros((nyears,len(ut.basin)))
    ttime = np.arange(nyears)
    months = np.arange(0,12)

    c = 0
    for f,fname in enumerate(fnames):
        ds = xr.open_dataset(fname)
        time = ds['time_centered'].values
        temp = ds['thetao'].values
        ds.close()

        year0 = int(fname[-27:-23])

        tb = np.zeros((len(ut.basin)))
        ny = int(len(time)/12)
        for y in np.arange(0,ny):
            for b,bas in enumerate(ut.basin):
                for m,mm in enumerate(months):
                    tbb = np.nansum(temp[m+12*y,:,:,:]*volume.sel(basin=bas))/np.nansum(volume.sel(basin=bas))
                    tb[b] += tbb*secs[m]
                tbas[c,b] = tb[b]/sum(secs)
            print(year0+y,f'{c:3.0f}',tbas[c,:],end='                                \r')
            tb = np.zeros((len(ut.basin)))
            c += 1

    temp2 = xr.DataArray(tbas,dims=('time','basin'),coords={'time':ttime,'basin':ut.basin},attrs={'unit':'degrees Celcius','long_name':'temperature time series per basin'})
    ds = xr.Dataset({'temp':temp2})
    savename = f'../data/temperature_ann_{run}.nc'
    ds.to_netcdf(savename,mode='w')
    ds.close()
    print('\n','Saved',savename) 

In [7]:
runs = ['tot2']
#runs = ['ctrl','hal2','half','tot2','totl','spin','quar']

for run in runs:
    #save_annual(run)
    save_monthly(run)

tot2 : 716 months =  59 y 8 m
Saved ../data/temperature_mon_tot2.nc                                                                                      
